In [1]:
import pandas as pd
import numpy as np
from sklearn import svm

In [2]:
train = pd.read_csv('features.train',sep='\s+',names=['digit','intensity','symmetry'])
test= pd.read_csv('features.test',sep='\s+',names=['digit','intensity','symmetry'])

# Ploynomial Kernels
## Question 2

In [3]:
def addoneversusall(train_data):
    for i in range(0,10):
        feature_name = str(i)
        train_data[feature_name]=train_data['digit'].apply(lambda x : 1 if x==i else -1)
    return train_data

In [4]:
def evaluate_Ein(train_data,features):
    for feature in features:
        train_label = train_data[feature]
        train_label = np.array(train_label)
        data = train_data[['intensity','symmetry']]
        data = np.array(data)
        clf = svm.SVC(C=0.01,kernel='poly',degree=2)
        clf.fit(data,train_label)
        print feature,(clf.predict(data)!=train_label).sum()     

In [5]:
train_data = addoneversusall(train)
evaluate_Ein(train_data,['0','2','4','6','8'])

0 1194
2 731
4 652
6 664
8 542


## Question 3

In [6]:
train_data = addoneversusall(train)
evaluate_Ein(train_data,['1','3','5','7','9'])

1 115
3 658
5 556
7 645
9 644


## Question 4

In [7]:
train_data = addoneversusall(train)
features = ['0','1']
for feature in features:
    train_label = train_data[feature]
    train_label = np.array(train_label)
    data = train_data[['intensity','symmetry']]
    data = np.array(data)
    clf = svm.SVC(C=0.01,kernel='poly',degree=2)
    clf.fit(data,train_label)
    print sum(clf.n_support_)

2390
536


## Question 5

In [8]:
test_data = test
test_data['1_5'] = 0
test_data['1_5'] =test_data['digit'].map({1:1,5:-1})
sub_test = test_data.dropna()

In [9]:
train_data['1_5'] = 0
train_data['1_5'] =train_data['digit'].map({1:1,5:-1})
sub_data = train_data.dropna()

In [10]:
for c in [0.001,0.01,0.1,1]:
    train_label = sub_data['1_5']
    train_label = np.array(train_label)
    data = sub_data[['intensity','symmetry']]
    data = np.array(data)
    clf = svm.SVC(C=c,kernel='poly',degree=2)
    clf.fit(data,train_label)
    print c,sum(clf.n_support_),'Ein',(clf.predict(data)!=train_label).sum() 

0.001 152 Ein 11
0.01 54 Ein 7
0.1 28 Ein 7
1 25 Ein 7


## Question 6

In [11]:
for c in [0.0001,0.001,0.01,1]:
    for q in [2,5]:
        train_label = sub_data['1_5']
        train_label = np.array(train_label)
        data = sub_data[['intensity','symmetry']]
        data = np.array(data)
        clf = svm.SVC(C=c,kernel='poly',degree=q)
        clf.fit(data,train_label)
        print c,q,sum(clf.n_support_),'Ein',(clf.predict(data)!=train_label).sum()
for q in [2,5]:
    train_label = sub_data['1_5']
    train_label = np.array(train_label)
    test_label = sub_test['1_5']
    test_label = np.array(test_label)
    data = sub_data[['intensity','symmetry']]
    data = np.array(data)
    test_data = sub_test[['intensity','symmetry']]
    test_data = np.array(test_data)
    clf = svm.SVC(C=1,kernel='poly',degree=q)
    clf.fit(data,train_label)
    print c,q,sum(clf.n_support_),'Eout',(clf.predict(test_data)!=test_label).sum()

0.0001 2 510 Ein 35
0.0001 5 42 Ein 10
0.001 2 152 Ein 11
0.001 5 28 Ein 8
0.01 2 54 Ein 7
0.01 5 26 Ein 7
1 2 25 Ein 7
1 5 26 Ein 7
1 2 25 Eout 8
1 5 26 Eout 7


# Cross Validation
## Question 7

In [12]:
from sklearn.model_selection import cross_val_score
train_data['1_5'] = 0
train_data['1_5'] =train_data['digit'].map({1:1,5:-1})
sub_data = train_data.dropna()

In [13]:
for c in [0.0001,0.001,0.01,0.1,1]:
    train_label = sub_data['1_5']
    train_label = np.array(train_label)
    data = sub_data[['intensity','symmetry']]
    data = np.array(data)
    clf = svm.SVC(C=c,kernel='poly',degree=2)
    scores = cross_val_score(clf,data,train_label,cv=10,scoring='f1')
    print c,1 - scores.mean()

0.0001 0.0184950823082
0.001 0.00594073893781
0.01 0.00348756218905
0.1 0.00348756218905
1 0.00399007475187


# RBF Kernel
## Question 9,10

In [14]:
for c in [0.01,1,100,1e4,1e6]:
    train_label = sub_data['1_5']
    train_label = np.array(train_label)
    test_label = sub_test['1_5']
    test_label = np.array(test_label)
    data = sub_data[['intensity','symmetry']]
    data = np.array(data)
    test_data = sub_test[['intensity','symmetry']]
    test_data = np.array(test_data)
    clf = svm.SVC(C=c,kernel='rbf')
    clf.fit(data,train_label)
    print c,'Ein',(clf.predict(data)!=train_label).sum(),'Eout',(clf.predict(test_data)!=test_label).sum()

0.01 Ein 6 Eout 9
1 Ein 7 Eout 9
100 Ein 5 Eout 8
10000.0 Ein 4 Eout 8
1000000.0 Ein 2 Eout 9
